In [5]:
import pandas as pd
import numpy as np
from string import punctuation

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

import os

In [6]:
df = pd.read_csv(f'/home/kar7mp5/Projects/MinGPT/data/article_contents/environment_8.csv')
df.head()

,Unnamed: 0,0
0,0,Skip linksSkip to Contentplay Live Show naviga...
1,1,play videoplay videoVideo Duration 02 minutes ...
2,2,A major bone of contention between negotiator...
3,3,A final round of talks is scheduled to be hel...
4,4,Keep reading list of 4 itemslist 1 of 4Japanes...


In [7]:
print('열의 개수: ', len(df.columns))
print(df.columns)

열의 개수:  2
Index(['Unnamed: 0', '0'], dtype='object')


In [8]:
print(df['0'].isnull().values.any())

False


In [9]:
headline = []
# 헤드라인의 값들을 리스트로 저장
headline.extend(list(df['0']))
headline[:]

['Skip linksSkip to Contentplay Live Show navigation menuNavigation menuNewsShow more news sectionsMiddle EastAfricaAsiaUS & CanadaLatin AmericaEuropeAsia PacificIsrael War on GazaFeaturesOpinionVideoMoreShow more sectionsEconomyUkraine warCoronavirusClimate CrisisInvestigationsInteractivesIn PicturesScience & TechnologySportPodcastsplay Live Click here to searchsearchEXPLAINERNews|EnvironmentInvisible plastic: Why banning plastic bags will never be enoughHow ordinary items like toothpaste and teabags could actually be damaging our environment',
 'play videoplay videoVideo Duration 02 minutes 37 seconds 02:37By\xa0Sarah ShamimPublished On 4 May 20244 May 2024This week, the fourth round of treaty talks by the Intergovernmental Negotiating Committee on Plastic Pollution concluded in Ottawa, Canada',
 ' A major bone of contention between negotiators from 175 countries is whether or not to limit the production of plastic, most of which is made from fossil fuels and chemicals and which caus

In [10]:
print('총 샘플의 개수 : {}'.format(len(headline)))

총 샘플의 개수 : 33


In [11]:
headline = [word for word in headline if word != "Unknown"]
print("노이즈값 제거 후 샘플의 개수 : {}".format(len(headline)))

노이즈값 제거 후 샘플의 개수 : 33


In [12]:
headline[:5]

['Skip linksSkip to Contentplay Live Show navigation menuNavigation menuNewsShow more news sectionsMiddle EastAfricaAsiaUS & CanadaLatin AmericaEuropeAsia PacificIsrael War on GazaFeaturesOpinionVideoMoreShow more sectionsEconomyUkraine warCoronavirusClimate CrisisInvestigationsInteractivesIn PicturesScience & TechnologySportPodcastsplay Live Click here to searchsearchEXPLAINERNews|EnvironmentInvisible plastic: Why banning plastic bags will never be enoughHow ordinary items like toothpaste and teabags could actually be damaging our environment',
 'play videoplay videoVideo Duration 02 minutes 37 seconds 02:37By\xa0Sarah ShamimPublished On 4 May 20244 May 2024This week, the fourth round of treaty talks by the Intergovernmental Negotiating Committee on Plastic Pollution concluded in Ottawa, Canada',
 ' A major bone of contention between negotiators from 175 countries is whether or not to limit the production of plastic, most of which is made from fossil fuels and chemicals and which caus

In [13]:
import re
from string import punctuation

def repreprocessing(raw_sentence):
    if isinstance(raw_sentence, float):
        # 부동 소수점일 경우 처리할 내용 추가
        return ""

    preproceseed_sentence = raw_sentence.encode("utf8").decode("ascii", "ignore")
    # 구두점 제거와 동시에 소문자화
    return "".join(word for word in preproceseed_sentence if word not in punctuation).lower()

preprocessed_headline = [repreprocessing(x) for x in headline if not isinstance(x, float)]
preprocessed_headline[:5]

['skip linksskip to contentplay live show navigation menunavigation menunewsshow more news sectionsmiddle eastafricaasiaus  canadalatin americaeuropeasia pacificisrael war on gazafeaturesopinionvideomoreshow more sectionseconomyukraine warcoronavirusclimate crisisinvestigationsinteractivesin picturesscience  technologysportpodcastsplay live click here to searchsearchexplainernewsenvironmentinvisible plastic why banning plastic bags will never be enoughhow ordinary items like toothpaste and teabags could actually be damaging our environment',
 'play videoplay videovideo duration 02 minutes 37 seconds 0237bysarah shamimpublished on 4 may 20244 may 2024this week the fourth round of treaty talks by the intergovernmental negotiating committee on plastic pollution concluded in ottawa canada',
 ' a major bone of contention between negotiators from 175 countries is whether or not to limit the production of plastic most of which is made from fossil fuels and chemicals and which causes pollution

In [14]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_headline)
vocab_size = len(tokenizer.word_index) + 1
print("단어 집합의 크기 : %d" % vocab_size)

단어 집합의 크기 : 479


In [15]:
sequences = list()

for sentence in preprocessed_headline:
    print(sentence)
    # 각 샘플에 대한 정수 인코딩
    encoded = tokenizer.texts_to_sequences([sentence])[0] 
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11]

skip linksskip to contentplay live show navigation menunavigation menunewsshow more news sectionsmiddle eastafricaasiaus  canadalatin americaeuropeasia pacificisrael war on gazafeaturesopinionvideomoreshow more sectionseconomyukraine warcoronavirusclimate crisisinvestigationsinteractivesin picturesscience  technologysportpodcastsplay live click here to searchsearchexplainernewsenvironmentinvisible plastic why banning plastic bags will never be enoughhow ordinary items like toothpaste and teabags could actually be damaging our environment
play videoplay videovideo duration 02 minutes 37 seconds 0237bysarah shamimpublished on 4 may 20244 may 2024this week the fourth round of treaty talks by the intergovernmental negotiating committee on plastic pollution concluded in ottawa canada
 a major bone of contention between negotiators from 175 countries is whether or not to limit the production of plastic most of which is made from fossil fuels and chemicals and which causes pollution after use

[[145, 146],
 [145, 146, 4],
 [145, 146, 4, 147],
 [145, 146, 4, 147, 69],
 [145, 146, 4, 147, 69, 148],
 [145, 146, 4, 147, 69, 148, 149],
 [145, 146, 4, 147, 69, 148, 149, 150],
 [145, 146, 4, 147, 69, 148, 149, 150, 151],
 [145, 146, 4, 147, 69, 148, 149, 150, 151, 70],
 [145, 146, 4, 147, 69, 148, 149, 150, 151, 70, 152],
 [145, 146, 4, 147, 69, 148, 149, 150, 151, 70, 152, 153]]

In [16]:
index_to_word = {}
for key, value in tokenizer.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print("빈도수 상위 582번 단어 : {}".format(index_to_word[5]))

빈도수 상위 582번 단어 : a


In [17]:
max_len = max(len(l) for l in sequences)
print("샘플의 최대 길이 : {}".format(max_len))

샘플의 최대 길이 : 77


In [18]:
sequences = pad_sequences(sequences, maxlen=max_len, padding="pre")
print(sequences[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0 145 146]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0 145 146   4]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0 145 146   4 147]]


In [19]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

print(X[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0 145]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0 145 146]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0 145 146   4]]


In [20]:
print(y[:3])

[146   4 147]


In [21]:
y = to_categorical(y, num_classes=vocab_size)

# Make the model

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [23]:
embedding_dim = 10
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200


31/31 - 3s - 100ms/step - accuracy: 0.0279 - loss: 6.0800
Epoch 2/200
31/31 - 1s - 46ms/step - accuracy: 0.0445 - loss: 5.7478
Epoch 3/200
31/31 - 1s - 47ms/step - accuracy: 0.0393 - loss: 5.6518
Epoch 4/200
31/31 - 1s - 44ms/step - accuracy: 0.0403 - loss: 5.6287
Epoch 5/200
31/31 - 1s - 45ms/step - accuracy: 0.0331 - loss: 5.6220
Epoch 6/200
31/31 - 1s - 44ms/step - accuracy: 0.0362 - loss: 5.6094
Epoch 7/200
31/31 - 1s - 45ms/step - accuracy: 0.0393 - loss: 5.5975
Epoch 8/200
31/31 - 1s - 44ms/step - accuracy: 0.0393 - loss: 5.5809
Epoch 9/200
31/31 - 1s - 46ms/step - accuracy: 0.0403 - loss: 5.5563
Epoch 10/200
31/31 - 1s - 42ms/step - accuracy: 0.0445 - loss: 5.5141
Epoch 11/200
31/31 - 1s - 44ms/step - accuracy: 0.0424 - loss: 5.4465
Epoch 12/200
31/31 - 1s - 44ms/step - accuracy: 0.0507 - loss: 5.3626
Epoch 13/200
31/31 - 1s - 44ms/step - accuracy: 0.0610 - loss: 5.2601
Epoch 14/200
31/31 - 1s - 43ms/step - accuracy: 0.0662 - loss: 5.1492
Epoch 15/200
31/31 - 1s - 44ms/step - ac

KeyboardInterrupt: 

In [24]:
model.save("./model/LSTM.keras")

In [25]:
import tensorflow as tf
model = tf.keras.models.load_model("./model/LSTM.keras")

In [41]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding="pre")

        # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word
        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [42]:
print(sentence_generation(model, tokenizer, "hi", 35))

hi are countries countries countries countries countries countries countries countries countries countries shamimpublished on countries countries countries countries countries countries canadalatin canadalatin americaeuropeasia americaeuropeasia americaeuropeasia americaeuropeasia ususer picturesscience usappsnewsletterschannel usappsnewsletterschannel usappsnewsletterschannel jazeera media jazeera jazeera jazeera


In [45]:
print(sentence_generation(model, tokenizer, "Trump, Donald J", 30))

Trump, Donald J are countries countries countries countries countries countries countries countries countries countries shamimpublished on countries countries countries countries countries countries canadalatin canadalatin americaeuropeasia americaeuropeasia americaeuropeasia americaeuropeasia ususer picturesscience usappsnewsletterschannel usappsnewsletterschannel usappsnewsletterschannel
